In [ ]:
import pandas as pd
import gurobipy as gp
import math
from datetime import datetime, timedelta

### Parameters

In [ ]:
product_type = "frozen" #frozen chilled ambient 
num_time_periods = 7
storage_type="standard" #standard constrained relaxed 
M = 2000

In [ ]:
# constants definitions
if(product_type=="frozen"):
    if(storage_type=="standard"):
        warehouse_volume = 30
    if(storage_type=="constrained"):
        warehouse_volume = 20
    if(storage_type=="relaxed"):
        warehouse_volume = 50
if(product_type=="chilled"):
    if(storage_type=="standard"):
        warehouse_volume = 250
    if(storage_type=="constrained"):
        warehouse_volume = 180
    if(storage_type=="relaxed"):
        warehouse_volume = 300
if(product_type=="ambient"):
    if(storage_type=="standard"):
        warehouse_volume = 700
    if(storage_type=="constrained"):
        warehouse_volume = 500
    if(storage_type=="relaxed"):
        warehouse_volume = 900
buffer_cost = 25
default_max_order = 10000
#max_order_exceed_multiplier = 0
max_order_exceed_penalty = 1000

### Schedule and Cost Variables

In [ ]:
schedules = {
    'construction': None,
    'operator1': None,
    'operator2': None
}

costs = {
    'construction': {
        'fixed':0,
        'per_tu':0,
        'clearance':0,
        'buffer':0,
        'max_order_penalty':0,
        'total':0
    },
    'operator1': {
        'fixed':0,
        'per_tu':0,
        'clearance':0,
        'buffer':0,
        'max_order_penalty':0,
        'total':0
    },
    'operator2': {
        'fixed':0,
        'per_tu':0,
        'clearance':0,
        'buffer':0,
        'max_order_penalty':0,
        'total':0
    }
}

### Data Importing

In [ ]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [ ]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]

In [ ]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [ ]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [ ]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [ ]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)

# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

### Construction

In [ ]:
def getCost(orders, order_date, cu_per_tu_item, order_q, disposal_cost):
        cost = 0
        if (orders[order_date]* cu_per_tu_item) - order_q > 0:
            cost = ((orders[order_date] * cu_per_tu_item) - order_q) * disposal_cost
        return cost   

def getMaxOrderPenalty(orders, order_date, variable_cost, maximum_order_quantity_item):
    max_order = 0
    if orders[order_date] > maximum_order_quantity_item: 
        max_order = ((orders[order_date]) - maximum_order_quantity_item) * max_order_exceed_penalty 
    return max_order 

def getTotalCosts(orders, order_date, cu_per_tu_item, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, order_q, expired_flag):
    tot_disposal = 0
    if expired_flag == True:
        tot_disposal = getCost(orders, order_date, cu_per_tu_item, order_q, disposal_cost)      
    tot_ordering = orders[order_date] * variable_cost
    tot_fixed = fixed_cost
    tot_max_order =  getMaxOrderPenalty(orders, order_date, variable_cost, maximum_order_quantity_tu_item)
    return tot_disposal, tot_ordering, tot_fixed, tot_max_order


In [ ]:
def silverMeal(item):
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    maximum_order_quantity_tu_item = maximum_order_quantity_tu[item]
    minimum_order_quantity_tu_item = minimum_order_quantity_tu[item]
    life = shelf_life[item]
    cost = 0
    tot_disposal = 0
    tot_fixed = 0
    tot_ordering = 0
    tot_max_order = 0
    penalty = 0
    orders = {}
    order_date = 0 
    run_complete = False
    obj_val = 0
    v = 0
    while(not run_complete):
  
        if (order_date %7 == 6):
            order_date = order_date + 1 
            continue   
        
        min_cost = float('inf')
        cum_demand = 0
        
        expired_flag = False

        for t in range(order_date, len(item_demand)):
            if(t%7==6 ):
                continue

            cum_demand += item_demand[t]
            order_q = cum_demand

            #if shelf life of an item exceeds the period
            if(t ==order_date+life):
                
                expired_flag = True
                if (t == len(item_demand)-1):
                    expired_flag = False
                order_q = cum_demand

                orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                if (math.ceil(order_q/cu_per_tu_item) < minimum_order_quantity_tu_item):
                    order_q = minimum_order_quantity_tu_item
                    orders[order_date] = order_q

                if v>0 and order_q -v > 0 :
                    order_q = order_q - v
                    orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                v = 0
                d, o, f, m = getTotalCosts(orders, order_date, cu_per_tu_item, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, order_q, expired_flag)
                tot_disposal += d
                tot_ordering += o
                tot_fixed += f
                tot_max_order += m
                order_date = t + 1
                break

            cost = fixed_cost + variable_cost * math.ceil(order_q/cu_per_tu_item) 
            if(math.ceil(order_q/cu_per_tu_item) > maximum_order_quantity_tu_item):
                penalty = (math.ceil(order_q/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_penalty
            avg_cost = (cost + penalty) / (t-order_date+1)
            
            #if the average cost is less now than later
            if(avg_cost <= min_cost):
                min_cost = (cost) / (t-order_date+1)
                if(t==len(item_demand)-1):
                    order_q = cum_demand
                    
                    orders[order_date] = math.ceil(order_q/cu_per_tu_item)

                    if (math.ceil(order_q/cu_per_tu_item) < minimum_order_quantity_tu_item):
                        order_q = minimum_order_quantity_tu_item
                        orders[order_date] = order_q
                    
                    if v>0  and  order_q - v > 0:
                        order_q = order_q - v
                        orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                    v = (orders[order_date]  * cu_per_tu_item) - order_q 
                    
                    d, o, f, m = getTotalCosts(orders, order_date, cu_per_tu_item, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, order_q, expired_flag)
                    tot_disposal += d
                    tot_ordering += o
                    tot_fixed += f
                    tot_max_order += m
                    run_complete = True
                    break

            # if end of time period is reached then just order till previoud demand           
            else:
                if(t==len(item_demand)-1):
                    order_q = cum_demand 
                else:
                    order_q = cum_demand - item_demand[t]
                
                orders[order_date] = math.ceil(order_q/cu_per_tu_item)

                if (math.ceil(order_q/cu_per_tu_item) < minimum_order_quantity_tu_item):
                    order_q = minimum_order_quantity_tu_item
                    orders[order_date] = order_q

                if v>0  and order_q - v > 0:
                    order_q = order_q - v
                    orders[order_date] = math.ceil(order_q/cu_per_tu_item)

                v = (orders[order_date]  * cu_per_tu_item) - order_q
                #print(orders[order_date] * cu_per_tu_item)

                d, o, f, m = getTotalCosts(orders, order_date, cu_per_tu_item, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, order_q, expired_flag)
                tot_disposal += d
                tot_ordering += o
                tot_fixed += f
                tot_max_order += m
                if(t==len(item_demand)-1):
                    order_date = t + 1
                    run_complete = True
                else:
                    order_date = t
                break
       
        obj_val += cost
        if(order_date > len(item_demand)-1):
            run_complete = True
            break
    tot_obj_val = tot_disposal +  tot_ordering + tot_fixed + tot_max_order     
    return (orders, tot_obj_val, tot_disposal, tot_ordering, tot_fixed, tot_max_order)

obj_val = 0 
vol = 0
tot_disposal = 0
tot_fixed = 0
tot_ordering = 0
tot_max_order = 0

schedule = {}
for item in demand.keys():
    orders, val, disposal, ordering, fixed, max_order_p = silverMeal(item) 
    obj_val += val
    tot_disposal += disposal
    tot_fixed += fixed
    tot_ordering += ordering
    tot_max_order += max_order_p
    for t in orders.keys():
        if orders[t] == 0:
            continue
        if item not in schedule.keys():
            schedule[item] = {}
        if t not in schedule[item].keys():
            schedule[item][t] = 0
        schedule[item][t] += orders[t]
            
    
#print(silverMeal('00ee8964'))
print(obj_val)
print('Ordering',tot_ordering)
print('Fixing ', tot_fixed)
print('Clearance ', tot_disposal)
print('Max Order ', tot_max_order)


In [ ]:
def bufferCostCalculation(schedule, demand):
    buf_cost = 0
    vol = 0
    volumes = {}

    for time in range(len(time_indexes)):
        for item in schedule:
            if time in schedule[item].keys():
                vol += schedule[item][time] * cu_per_tu[item] * volume_per_cu[item]

        if time > 0:
            for item in demand.keys():
                if time in demand[item].keys():
                    vol -= demand[item][time] * volume_per_cu[item]

        volumes[time] = vol
        if vol > warehouse_volume:

            buf_cost += (vol - warehouse_volume) * buffer_cost

    return buf_cost

In [ ]:
buf_cost = bufferCostCalculation(schedule, demand) 
schedules['construction'] = schedule.copy()
costs['construction']['total'] = obj_val + buf_cost
costs['construction']['buffer'] = buf_cost
costs['construction']['clearance'] = tot_disposal
costs['construction']['fixed'] = tot_fixed
costs['construction']['per_tu'] = tot_ordering
costs['construction']['max_order_penalty'] = tot_max_order

costs['operator1'] = costs['construction'].copy()

print("Construction heuristics costs: ",costs['construction'])

### Improvement

#### Operator 1

In [ ]:
def getVolumes(schedule):
    vol = 0
    volumes = {}

    for time in range(len(time_indexes)):
        for item in schedule:
            if time in schedule[item].keys():
                vol += schedule[item][time] * cu_per_tu[item] * volume_per_cu[item]

        if time > 0:
            for item in demand.keys():
                if time in demand[item].keys():
                    vol -= demand[item][time] * volume_per_cu[item]

        volumes[time] = vol
    return volumes

def operator1(schedule, obj_val, costs, schedules):
    max_improvement = math.inf
    cost = obj_val
    modified_schedule = {}
    volumes = getVolumes(schedule)

    buffer_penalty_tot = 0
    max_order_penalty_tot = 0
    fixed_ordering_cost_tot = 0 
    schedule_imp = schedule 
    flag = True

    count = 0
    iterator = 0
    MAX_ITER = 100
    MAX_REPEAT = 3
    prev_max_improvement = 0

    while count < MAX_REPEAT and iterator < MAX_ITER:
        for item in schedule_imp:
            buffer_penalty = 0
            max_order_penalty = 0 
            modified_orders = {}

            # Check if the item has a second order date
            if len(schedule_imp[item]) > 1:
                order_days = list(schedule_imp[item].keys())
                for i in range (0,len(order_days)-1, 2):
                    day1 = i
                    day2 = i + 1
                    if len(schedule_imp[item])%2 != 0:
                        day3 = i + 2
                        
                        if (order_days[day3] - order_days[day2] < shelf_life[item]):
                            flag = True 
                        else: 
                            modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                            modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
                            modified_orders[order_days[day3]] = schedule_imp[item][order_days[day3]]
                            continue

                    if order_days[day2] - order_days[day1] < shelf_life[item] and (len(time_indexes)-1 - order_days[day2] < shelf_life[item]) and flag:
                        tot_orders = schedule_imp[item][order_days[day1]]+ schedule_imp[item][order_days[day2]]
                        vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
                        if vol + volumes[day1] > warehouse_volume:
                            buffer_penalty += buffer_cost * (vol + volumes[day1] - warehouse_volume)
                        if tot_orders > maximum_order_quantity_tu[item]:
                            max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item]) * max_order_exceed_penalty 
                        delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
                        if delta < 0:
                            fixed_ordering_cost_tot += ordering_cost_fixed[item] 
                            buffer_penalty_tot += buffer_penalty
                            max_order_penalty_tot += max_order_penalty
                            cost += delta
                            modified_orders[order_days[day1]] = tot_orders
                            volumes[order_days[day1]] += vol
                        else:
                            modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                            modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]

                    else:
                        modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                        modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
                    
                    if len(schedule_imp[item])%2 != 0:
                        modified_orders[order_days[-1]] = schedule_imp[item][order_days[-1]]
                
                
                modified_schedule[item] = modified_orders
            
            else:
                if len(schedule[item]) == 1 : 
                    modified_schedule[item] = schedule_imp[item]
                else:
                    continue


            if cost!=0 and cost < max_improvement:
                max_improvement = cost
        
        # print("Buffer penalty paid", buffer_penalty_tot)
        # print("max order penalty paid", max_order_penalty_tot)
        # print("fixed cost reduced by", fixed_ordering_cost_tot)
        schedule_imp = modified_schedule
        obj_val = max_improvement
        iterator +=1

        # print(max_improvement)

        if(max_improvement==prev_max_improvement):
            count+=1
        prev_max_improvement = max_improvement

    
    costs['operator1']['buffer'] += buffer_penalty_tot
    costs['operator1']['max_order_penalty'] += max_order_penalty_tot
    costs['operator1']['fixed'] -= fixed_ordering_cost_tot
    costs['operator1']['total'] = obj_val

    costs['operator2'] = costs['operator1'].copy()

    schedules['operator1'] = schedule_imp.copy()

    return None

#### Operator 2

In [ ]:
def transposeScheduleToDayFirst(schedule):
    result_dict = {}
    for day, articles in schedule.items():
        for article, quantity in articles.items():
            if article in result_dict:
                result_dict[article][day] = quantity
            else:
                result_dict[article] = {day: quantity}
    return result_dict

def transposeScheduleToItemFirst(schedule):
    result_dict = {}
    for article, days in schedule.items():
        for day, quantity in days.items():
            if day in result_dict:
                result_dict[day][article] = quantity
            else:
                result_dict[day] = {article: quantity}
    return result_dict

def operator2(schedule, obj_val, costs, schedules):
    global ordering_cost_per_tu_saved
    ordering_cost_per_tu_saved = 0
    global new_clearance_cost
    new_clearance_cost = 0

    # Function removes expired quantities from inventory.
    # 
    # If I ever have to throw away equal to or more than 
    # a TU's worth, then I can just reduce order.
    # We can save on per TU cost (done in function)
    # We can save on fixed cost (done in parent)
    # We can save on buffer cost (done in parent)
    # We can save on max order penalty (done in parent)
    # We can save on clearance cost (done in function)
    def removeExpired(inv, life, dem):
        global ordering_cost_per_tu_saved
        global new_clearance_cost
        
        tu_saved = 0
        # if inventory has items from days which are now too old
        # life + 1 because:
        # if an item with shelf life 3 is ordered on day 0,
        # it can stay in shelf until day 3 (0-1,1-2,2-3) and can be
        # sold on day 3
        if(len(inv)>life+1):
            amount_cleared = inv[0]

            tu_saved = amount_cleared//cu_per_tu[item]
            # ordering cost reduced here
            ordering_cost_per_tu_saved += tu_saved * ordering_cost_per_tu[item]

            amount_inevitably_cleared = amount_cleared%cu_per_tu[item]
            # clearance cost recalculated here
            new_clearance_cost += amount_inevitably_cleared*clearing_cost_per_cu[item]
            
            # remove days which are now too old
            inv = inv[-life:]
        
        return inv, tu_saved

    def addDay(inv, ordered):
        inv.append(ordered)
        return inv
    
    # Satisfying current demand from quantity in inventory
    def satisfyFromInventory(inv, dem):
        for day in range(len(inv)):
            if(dem==0):
                break
            if(inv[day] <= dem):
                dem -= inv[day]
                inv[day] = 0
            else:
                inv[day] -= dem
                dem = 0
        return inv

    def updateInventory(inv, item, t, ordered):
        life = shelf_life[item]
        # set demand as 0 if there is no demand for an item on a day
        if (item in demand[t].keys()):
            dem = demand[t][item]
        else:
            dem = 0
        # first the orders of the new day are added to the inventory
        # and then the total inventory is used to satisfy the demand
        # This would mean that the demand cannot go unsatisfied from
        # the items in the inventory
        inv = addDay(inv, ordered)
        inv, tu_saved = removeExpired(inv,life,item)
        inv = satisfyFromInventory(inv, dem)
        return inv, tu_saved
    
    def calculateMaxOrderPenalty(schedule):
        max_order_penalty = 0
        for day in schedule.keys():
            for item in schedule[day].keys():
                if(schedule[day][item] > maximum_order_quantity_tu[item]):
                    max_order_penalty += (schedule[day][item] - maximum_order_quantity_tu[item]) * max_order_exceed_penalty 
        return max_order_penalty
    
    def calculateFixedOrderingCost(schedule):
        fixed_ordering_cost = 0
        for day in schedule.keys():
            for item in schedule[day].keys():
                if(schedule[day][item] > 0):
                    fixed_ordering_cost += ordering_cost_fixed[item]
        return fixed_ordering_cost

    def endClearance(schedule, item, inv):
        per_tu_cost_saved = 0

        cu_surplus = sum(inv)
        tu_surplus = cu_surplus // cu_per_tu[item]
        cu_disposed = cu_surplus % cu_per_tu[item]
        for day in range(len(schedule)-1,-1,-1):
            if(day in schedule.keys()):
                if(tu_surplus <= 0):
                    break
                if(schedule[day] >= tu_surplus and schedule[day] - tu_surplus >= minimum_order_quantity_tu[item]):
                    schedule[day] -= tu_surplus
                    break
                else:
                    tu_surplus -= schedule[day]
                    schedule[day] = 0
        cu_disposed += tu_surplus * cu_per_tu[item]
        per_tu_cost_saved += tu_surplus * ordering_cost_per_tu[item]
        clearance_cost = cu_disposed * clearing_cost_per_cu[item]
        return schedule, clearance_cost, per_tu_cost_saved

    # dictionary of format {'item':['q left on earliest possible','q left on earliest possible + 1',...,'q left today']}
    inventory = {}

    for t in time_indexes:
        for item in articles:

            if item not in inventory.keys():
                inventory[item] = []

            # if there was no order for an item in a particular day, then the order was 0
            if(t in schedule.keys() and item in schedule[t].keys()):
                ordered = schedule[t][item] * cu_per_tu[item]
            else:
                ordered = 0

            # update the inventory according to what has been ordered today for item
            inventory[item],tu_saved = updateInventory(inventory[item], item, t, ordered)

            # Actual modification of the schedule.
            # Removing excess TUs from the schedule. On day t, we remove the excess from day t-shelf life
            if(t-shelf_life[item] in schedule.keys() and item in schedule[t-shelf_life[item]].keys()):
                if(t>=shelf_life[item]):
                    # If we are trying to save too much (that order goes below min order)
                    # we must only save enough to keep the order above min
                    # and remove excess reduction (happening in the removeExpired function)
                    # from the order_cost_per_tu_saved
                    if(schedule[t-shelf_life[item]][item] - tu_saved > 0 and schedule[t-shelf_life[item]][item] - tu_saved < minimum_order_quantity_tu[item]):
                        diff = minimum_order_quantity_tu[item] - (schedule[t-shelf_life[item]][item] - tu_saved)
                        schedule[t-shelf_life[item]][item] = minimum_order_quantity_tu[item]
                        ordering_cost_per_tu_saved -= diff * ordering_cost_per_tu[item]
                    else:
                        schedule[t-shelf_life[item]][item] -= tu_saved

    # Removing excess orders that never get cleared because time period
    # ends before their expiry (items with large shelf life)
    schedule = transposeScheduleToItemFirst(schedule)
    for item in schedule.keys():
        schedule[item], end_clearance_cost, per_tu_cost_saved = endClearance(schedule[item], item, inventory[item])
        ordering_cost_per_tu_saved += per_tu_cost_saved
        new_clearance_cost += end_clearance_cost
    schedule = transposeScheduleToDayFirst(schedule)

    # Recalculating fixed ordering costs since after saving TUs,
    # we might now have days when we are no longer making any orders
    # for an item that we used to order earlier
    new_fixed_ordering_cost = calculateFixedOrderingCost(schedule)

    # Recalculating max order penalty since after saving TUs,
    # we might now have days when we are no longer exceeding max limit
    # for an item that we used excees earlier
    new_max_order_penalty = calculateMaxOrderPenalty(schedule)

    costs['operator2']['clearance'] = new_clearance_cost
    costs['operator2']['max_order_penalty'] = new_max_order_penalty
    costs['operator2']['per_tu'] -= ordering_cost_per_tu_saved
    costs['operator2']['fixed'] = new_fixed_ordering_cost
    costs['operator2']['total'] = costs['operator2']['fixed'] + costs['operator2']['per_tu'] + costs['operator2']['buffer'] + costs['operator2']['clearance'] + costs['operator2']['max_order_penalty']

    schedules['operator2'] = transposeScheduleToItemFirst(schedule.copy())

In [ ]:
def transposeDemand(demand):
    result_dict = {}
    for item, days in demand.items():
        for day, quantity in days.items():
            if day in result_dict:
                result_dict[day][item] = quantity
            else:
                result_dict[day] = {item:quantity}
    return result_dict

def transposeBackDemand(demand):
    result_dict = {}
    for day, items in demand.items():
        for item, quantity in items.items():
            if item in result_dict:
                result_dict[item][day] = quantity
            else:
                result_dict[item] = {day:quantity}
    return result_dict


In [ ]:
curr_costs = None
new_obj_val = 0

ITERATION_LIMIT = 10
iteration_counter = 0

operator1(schedules['construction'], costs['construction']['total'], costs, schedules)
curr_obj_val = costs['operator1']['total']
curr_costs = costs['operator1']
print(curr_costs)
while(True):
    iteration_counter += 1

    # Operator2
    demand = transposeDemand(demand)
    operator2(transposeScheduleToDayFirst(schedules['operator1']), costs['operator1']['total'], costs, schedules)
    demand = transposeBackDemand(demand)
    new_obj_val = costs['operator2']['total']
    if(new_obj_val < curr_obj_val and iteration_counter < ITERATION_LIMIT):
        curr_obj_val = new_obj_val
        curr_costs = costs['operator2']
    else:
        break

    # Operator1
    operator1(schedules['operator2'], costs['operator2']['total'], costs, schedules)
    new_obj_val = costs['operator1']['total']
    if(new_obj_val < curr_obj_val and iteration_counter < ITERATION_LIMIT):
        curr_obj_val = new_obj_val
        curr_costs = costs['operator1']
    else:
        break
print("Final costs: ", curr_costs)